In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{asc, col, desc}
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType}

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.{asc, col, desc}
import org.apache.spark.sql.types.{IntegerType, StringType, StructField, StructType}


In [3]:
val sparkSession = SparkSession
    .builder()
    .master("local[*]")
    .appName("curso")
    .config("spark.some.config.option", "some-value")
    .getOrCreate()

sparkSession: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1cc7f32c


In [14]:
val cus = sparkSession.read.option("header", "true").csv("customers")

cus: org.apache.spark.sql.DataFrame = [customer_id: string, customer_fname: string ... 7 more fields]


In [15]:
cus.show()

+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|customer_id|customer_fname|customer_lname|customer_email|customer_password|     customer_street|customer_city|customer_state|customer_zipcode|
+-----------+--------------+--------------+--------------+-----------------+--------------------+-------------+--------------+----------------+
|          1|       Richard|     Hernandez|     XXXXXXXXX|        XXXXXXXXX|  6303 Heather Plaza|  Brownsville|            TX|           78521|
|          2|          Mary|       Barrett|     XXXXXXXXX|        XXXXXXXXX|9526 Noble Embers...|    Littleton|            CO|           80126|
|          3|           Ann|         Smith|     XXXXXXXXX|        XXXXXXXXX|3422 Blue Pioneer...|       Caguas|            PR|           00725|
|          4|          Mary|         Jones|     XXXXXXXXX|        XXXXXXXXX|  8324 Little Common|   San Marcos|            CA|          

In [9]:
val esq = StructType(Array(
    StructField("order_id", IntegerType, true),
    StructField("order_date", StringType, true),
    StructField("order_customer_id", IntegerType, true),
    StructField("order_status", StringType, true)
  ))

esq: org.apache.spark.sql.types.StructType = StructType(StructField(order_id,IntegerType,true), StructField(order_date,StringType,true), StructField(order_customer_id,IntegerType,true), StructField(order_status,StringType,true))


In [11]:
val ord = sparkSession.read.option("header", "true").schema(esq).csv("orders")

ord: org.apache.spark.sql.DataFrame = [order_id: int, order_date: string ... 2 more fields]


In [12]:
ord.show()

+--------+--------------------+-----------------+---------------+
|order_id|          order_date|order_customer_id|   order_status|
+--------+--------------------+-----------------+---------------+
|       2|2013-07-25 00:00:...|              256|PENDING_PAYMENT|
|       3|2013-07-25 00:00:...|            12111|       COMPLETE|
|       4|2013-07-25 00:00:...|             8827|         CLOSED|
|       5|2013-07-25 00:00:...|            11318|       COMPLETE|
|       6|2013-07-25 00:00:...|             7130|       COMPLETE|
|       7|2013-07-25 00:00:...|             4530|       COMPLETE|
|       8|2013-07-25 00:00:...|             2911|     PROCESSING|
|       9|2013-07-25 00:00:...|             5657|PENDING_PAYMENT|
|      10|2013-07-25 00:00:...|             5648|PENDING_PAYMENT|
|      11|2013-07-25 00:00:...|              918| PAYMENT_REVIEW|
|      12|2013-07-25 00:00:...|             1837|         CLOSED|
|      13|2013-07-25 00:00:...|             9149|PENDING_PAYMENT|
|      14|

In [16]:
val join = ord.groupBy(col("order_customer_id")).count().where(col("count").gt(5))
    .join(cus, ord.col("order_customer_id") === cus.col("customer_id") , "inner" )
    .select("customer_fname","customer_lname", "count" )
    .filter(col("customer_fname").startsWith("M"))
    .orderBy(desc("count"))

join: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [customer_fname: string, customer_lname: string ... 1 more field]


In [17]:
join.show()

+--------------+--------------+-----+
|customer_fname|customer_lname|count|
+--------------+--------------+-----+
|          Mary|          Frye|   16|
|          Mary|       Griffin|   16|
|          Mary|          Rios|   16|
|          Mary|          Rios|   16|
|          Mary|       Griffin|   16|
|          Mary|          Frye|   16|
|          Mary|           Cox|   15|
|          Mary|         Smith|   15|
|          Mary|           Cox|   15|
|          Mary|         Smith|   15|
|          Mary|          Mata|   15|
|          Mary|          Mata|   15|
|          Mary|         Cline|   14|
|          Mary|        Butler|   14|
|          Mary|         Cline|   14|
|          Mary|         Smith|   14|
|          Mary|         Smith|   14|
|          Mary|        Butler|   14|
|          Mary|         Smith|   14|
|          Mary|         Smith|   14|
+--------------+--------------+-----+
only showing top 20 rows

